In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset, TrainCleanSoundsDataset, PsychophysicsCleanSoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'

# Parameters

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
train_datafile = f'{engram_dir}training_dataset_random_order.hdf5'

Device: cuda:0


In [4]:
!nvidia-smi

Sat Aug  6 18:07:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:60:00.0 Off |                  N/A |
|  0%   25C    P8     1W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [8]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up train/test dataset

In [9]:
train_dataset = CleanSoundsDataset(train_datafile, .9)
test_dataset = CleanSoundsDataset(train_datafile, .9, train = False)

In [10]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [11]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Training Epoch: 1 [50/49669]	Loss: 0.4564
Training Epoch: 1 [100/49669]	Loss: 0.6180
Training Epoch: 1 [150/49669]	Loss: 0.2616
Training Epoch: 1 [200/49669]	Loss: 0.2126
Training Epoch: 1 [250/49669]	Loss: 0.3171
Training Epoch: 1 [300/49669]	Loss: 0.2529
Training Epoch: 1 [350/49669]	Loss: 0.1540
Training Epoch: 1 [400/49669]	Loss: 0.1414
Training Epoch: 1 [450/49669]	Loss: 0.1964
Training Epoch: 1 [500/49669]	Loss: 0.2026
Training Epoch: 1 [550/49669]	Loss: 0.1363
Training Epoch: 1 [600/49669]	Loss: 0.0996
Training Epoch: 1 [650/49669]	Loss: 0.1165
Training Epoch: 1 [700/49669]	Loss: 0.1430
Training Epoch: 1 [750/49669]	Loss: 0.1281
Training Epoch: 1 [800/49669]	Loss: 0.0880
Training Epoch: 1 [850/49669]	Loss: 0.0811
Training Epoch: 1 [900/49669]	Loss: 0.0898
Training Epoch: 1 [950/49669]	Loss: 0.1011
Training Epoch: 1 [1000/49669]	Loss: 0.0921
Training Epoch: 1 [1050/49669]	Loss: 0.0689
Training Epoch: 1 [1100/49669]	Loss: 0.0621
Training Epoch: 1 [1150/49669]	Loss: 0.0736
Training

Training Epoch: 1 [9400/49669]	Loss: 0.0138
Training Epoch: 1 [9450/49669]	Loss: 0.0143
Training Epoch: 1 [9500/49669]	Loss: 0.0129
Training Epoch: 1 [9550/49669]	Loss: 0.0122
Training Epoch: 1 [9600/49669]	Loss: 0.0131
Training Epoch: 1 [9650/49669]	Loss: 0.0132
Training Epoch: 1 [9700/49669]	Loss: 0.0138
Training Epoch: 1 [9750/49669]	Loss: 0.0133
Training Epoch: 1 [9800/49669]	Loss: 0.0136
Training Epoch: 1 [9850/49669]	Loss: 0.0129
Training Epoch: 1 [9900/49669]	Loss: 0.0123
Training Epoch: 1 [9950/49669]	Loss: 0.0131
Training Epoch: 1 [10000/49669]	Loss: 0.0133
Training Epoch: 1 [10050/49669]	Loss: 0.0137
Training Epoch: 1 [10100/49669]	Loss: 0.0132
Training Epoch: 1 [10150/49669]	Loss: 0.0124
Training Epoch: 1 [10200/49669]	Loss: 0.0130
Training Epoch: 1 [10250/49669]	Loss: 0.0134
Training Epoch: 1 [10300/49669]	Loss: 0.0132
Training Epoch: 1 [10350/49669]	Loss: 0.0118
Training Epoch: 1 [10400/49669]	Loss: 0.0138
Training Epoch: 1 [10450/49669]	Loss: 0.0125
Training Epoch: 1 [105

Training Epoch: 1 [18550/49669]	Loss: 0.0096
Training Epoch: 1 [18600/49669]	Loss: 0.0101
Training Epoch: 1 [18650/49669]	Loss: 0.0095
Training Epoch: 1 [18700/49669]	Loss: 0.0093
Training Epoch: 1 [18750/49669]	Loss: 0.0096
Training Epoch: 1 [18800/49669]	Loss: 0.0088
Training Epoch: 1 [18850/49669]	Loss: 0.0094
Training Epoch: 1 [18900/49669]	Loss: 0.0088
Training Epoch: 1 [18950/49669]	Loss: 0.0094
Training Epoch: 1 [19000/49669]	Loss: 0.0089
Training Epoch: 1 [19050/49669]	Loss: 0.0094
Training Epoch: 1 [19100/49669]	Loss: 0.0087
Training Epoch: 1 [19150/49669]	Loss: 0.0095
Training Epoch: 1 [19200/49669]	Loss: 0.0089
Training Epoch: 1 [19250/49669]	Loss: 0.0093
Training Epoch: 1 [19300/49669]	Loss: 0.0088
Training Epoch: 1 [19350/49669]	Loss: 0.0087
Training Epoch: 1 [19400/49669]	Loss: 0.0096
Training Epoch: 1 [19450/49669]	Loss: 0.0100
Training Epoch: 1 [19500/49669]	Loss: 0.0087
Training Epoch: 1 [19550/49669]	Loss: 0.0088
Training Epoch: 1 [19600/49669]	Loss: 0.0101
Training E

Training Epoch: 1 [27700/49669]	Loss: 0.0069
Training Epoch: 1 [27750/49669]	Loss: 0.0074
Training Epoch: 1 [27800/49669]	Loss: 0.0073
Training Epoch: 1 [27850/49669]	Loss: 0.0077
Training Epoch: 1 [27900/49669]	Loss: 0.0072
Training Epoch: 1 [27950/49669]	Loss: 0.0075
Training Epoch: 1 [28000/49669]	Loss: 0.0077
Training Epoch: 1 [28050/49669]	Loss: 0.0074
Training Epoch: 1 [28100/49669]	Loss: 0.0076
Training Epoch: 1 [28150/49669]	Loss: 0.0068
Training Epoch: 1 [28200/49669]	Loss: 0.0070
Training Epoch: 1 [28250/49669]	Loss: 0.0077
Training Epoch: 1 [28300/49669]	Loss: 0.0065
Training Epoch: 1 [28350/49669]	Loss: 0.0080
Training Epoch: 1 [28400/49669]	Loss: 0.0072
Training Epoch: 1 [28450/49669]	Loss: 0.0078
Training Epoch: 1 [28500/49669]	Loss: 0.0072
Training Epoch: 1 [28550/49669]	Loss: 0.0078
Training Epoch: 1 [28600/49669]	Loss: 0.0070
Training Epoch: 1 [28650/49669]	Loss: 0.0068
Training Epoch: 1 [28700/49669]	Loss: 0.0070
Training Epoch: 1 [28750/49669]	Loss: 0.0071
Training E

...

Training Epoch: 5 [3200/49669]	Loss: 0.0045
Training Epoch: 5 [3250/49669]	Loss: 0.0047
Training Epoch: 5 [3300/49669]	Loss: 0.0045
Training Epoch: 5 [3350/49669]	Loss: 0.0043
Training Epoch: 5 [3400/49669]	Loss: 0.0040
Training Epoch: 5 [3450/49669]	Loss: 0.0042
Training Epoch: 5 [3500/49669]	Loss: 0.0042
Training Epoch: 5 [3550/49669]	Loss: 0.0043
Training Epoch: 5 [3600/49669]	Loss: 0.0043
Training Epoch: 5 [3650/49669]	Loss: 0.0041
Training Epoch: 5 [3700/49669]	Loss: 0.0041
Training Epoch: 5 [3750/49669]	Loss: 0.0045
Training Epoch: 5 [3800/49669]	Loss: 0.0043
Training Epoch: 5 [3850/49669]	Loss: 0.0040
Training Epoch: 5 [3900/49669]	Loss: 0.0041
Training Epoch: 5 [3950/49669]	Loss: 0.0037
Training Epoch: 5 [4000/49669]	Loss: 0.0043
Training Epoch: 5 [4050/49669]	Loss: 0.0039
Training Epoch: 5 [4100/49669]	Loss: 0.0044
Training Epoch: 5 [4150/49669]	Loss: 0.0041
Training Epoch: 5 [4200/49669]	Loss: 0.0038
Training Epoch: 5 [4250/49669]	Loss: 0.0041
Training Epoch: 5 [4300/49669]	L

Training Epoch: 5 [12500/49669]	Loss: 0.0042
Training Epoch: 5 [12550/49669]	Loss: 0.0040
Training Epoch: 5 [12600/49669]	Loss: 0.0041
Training Epoch: 5 [12650/49669]	Loss: 0.0041
Training Epoch: 5 [12700/49669]	Loss: 0.0046
Training Epoch: 5 [12750/49669]	Loss: 0.0046
Training Epoch: 5 [12800/49669]	Loss: 0.0053
Training Epoch: 5 [12850/49669]	Loss: 0.0060
Training Epoch: 5 [12900/49669]	Loss: 0.0080
Training Epoch: 5 [12950/49669]	Loss: 0.0110
Training Epoch: 5 [13000/49669]	Loss: 0.0144
Training Epoch: 5 [13050/49669]	Loss: 0.0151
Training Epoch: 5 [13100/49669]	Loss: 0.0138
Training Epoch: 5 [13150/49669]	Loss: 0.0106
Training Epoch: 5 [13200/49669]	Loss: 0.0077
Training Epoch: 5 [13250/49669]	Loss: 0.0059
Training Epoch: 5 [13300/49669]	Loss: 0.0047
Training Epoch: 5 [13350/49669]	Loss: 0.0042
Training Epoch: 5 [13400/49669]	Loss: 0.0044
Training Epoch: 5 [13450/49669]	Loss: 0.0049
Training Epoch: 5 [13500/49669]	Loss: 0.0052
Training Epoch: 5 [13550/49669]	Loss: 0.0054
Training E

Training Epoch: 5 [21650/49669]	Loss: 0.0042
Training Epoch: 5 [21700/49669]	Loss: 0.0040
Training Epoch: 5 [21750/49669]	Loss: 0.0040
Training Epoch: 5 [21800/49669]	Loss: 0.0041
Training Epoch: 5 [21850/49669]	Loss: 0.0040
Training Epoch: 5 [21900/49669]	Loss: 0.0041
Training Epoch: 5 [21950/49669]	Loss: 0.0039
Training Epoch: 5 [22000/49669]	Loss: 0.0042
Training Epoch: 5 [22050/49669]	Loss: 0.0040
Training Epoch: 5 [22100/49669]	Loss: 0.0044
Training Epoch: 5 [22150/49669]	Loss: 0.0042
Training Epoch: 5 [22200/49669]	Loss: 0.0043
Training Epoch: 5 [22250/49669]	Loss: 0.0039
Training Epoch: 5 [22300/49669]	Loss: 0.0042
Training Epoch: 5 [22350/49669]	Loss: 0.0041
Training Epoch: 5 [22400/49669]	Loss: 0.0042
Training Epoch: 5 [22450/49669]	Loss: 0.0039
Training Epoch: 5 [22500/49669]	Loss: 0.0042
Training Epoch: 5 [22550/49669]	Loss: 0.0040
Training Epoch: 5 [22600/49669]	Loss: 0.0040
Training Epoch: 5 [22650/49669]	Loss: 0.0039
Training Epoch: 5 [22700/49669]	Loss: 0.0039
Training E

Training Epoch: 5 [30800/49669]	Loss: 0.0069
Training Epoch: 5 [30850/49669]	Loss: 0.0077
Training Epoch: 5 [30900/49669]	Loss: 0.0083
Training Epoch: 5 [30950/49669]	Loss: 0.0071
Training Epoch: 5 [31000/49669]	Loss: 0.0051
Training Epoch: 5 [31050/49669]	Loss: 0.0041
Training Epoch: 5 [31100/49669]	Loss: 0.0048
Training Epoch: 5 [31150/49669]	Loss: 0.0057
Training Epoch: 5 [31200/49669]	Loss: 0.0063
Training Epoch: 5 [31250/49669]	Loss: 0.0061
Training Epoch: 5 [31300/49669]	Loss: 0.0046
Training Epoch: 5 [31350/49669]	Loss: 0.0041
Training Epoch: 5 [31400/49669]	Loss: 0.0041
Training Epoch: 5 [31450/49669]	Loss: 0.0048
Training Epoch: 5 [31500/49669]	Loss: 0.0051
Training Epoch: 5 [31550/49669]	Loss: 0.0053
Training Epoch: 5 [31600/49669]	Loss: 0.0051
Training Epoch: 5 [31650/49669]	Loss: 0.0044
Training Epoch: 5 [31700/49669]	Loss: 0.0037
Training Epoch: 5 [31750/49669]	Loss: 0.0040
Training Epoch: 5 [31800/49669]	Loss: 0.0046
Training Epoch: 5 [31850/49669]	Loss: 0.0042
Training E

Training Epoch: 5 [39950/49669]	Loss: 0.0038
Training Epoch: 5 [40000/49669]	Loss: 0.0045
Training Epoch: 5 [40050/49669]	Loss: 0.0042
Training Epoch: 5 [40100/49669]	Loss: 0.0040
Training Epoch: 5 [40150/49669]	Loss: 0.0041
Training Epoch: 5 [40200/49669]	Loss: 0.0039
Training Epoch: 5 [40250/49669]	Loss: 0.0041
Training Epoch: 5 [40300/49669]	Loss: 0.0043
Training Epoch: 5 [40350/49669]	Loss: 0.0045
Training Epoch: 5 [40400/49669]	Loss: 0.0046
Training Epoch: 5 [40450/49669]	Loss: 0.0050
Training Epoch: 5 [40500/49669]	Loss: 0.0053
Training Epoch: 5 [40550/49669]	Loss: 0.0063
Training Epoch: 5 [40600/49669]	Loss: 0.0070
Training Epoch: 5 [40650/49669]	Loss: 0.0088
Training Epoch: 5 [40700/49669]	Loss: 0.0108
Training Epoch: 5 [40750/49669]	Loss: 0.0117
Training Epoch: 5 [40800/49669]	Loss: 0.0110
Training Epoch: 5 [40850/49669]	Loss: 0.0091
Training Epoch: 5 [40900/49669]	Loss: 0.0071
Training Epoch: 5 [40950/49669]	Loss: 0.0057
Training Epoch: 5 [41000/49669]	Loss: 0.0046
Training E